In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# ==========================================
# PHASE 1: DATA INGESTION & FEATURE EXTRACTION
# ==========================================
def load_and_extract_his(base_directory):
    """
    Parses the PHME 2026 dataset folders and extracts multiple HIs:
    HI_MAX (Position), HI_RMS_CUR (Currents), and HI_RMS_VOL (Voltages).
    Returns a list of 2D arrays (shape: [cycles, 3 features]).
    """
    all_trajectories = []
    
    # Iterate through all folders in the directory
    folder_paths = sorted(glob.glob(os.path.join(base_directory, "Train_*")))
    
    for folder in folder_paths:
        trajectory_features = []
        
        # Process the closing phase
        closing_files = sorted(glob.glob(os.path.join(folder, "*_Closing.csv")))
        
        for file in closing_files:
            # Load the CSV
            df = pd.read_csv(file, sep=None, engine='python') 
            
            # 1. Extract HI_MAX from POS_FBK (Column 3 -> index 2)
            hi_max = df.iloc[:, 2].max()
            
            # 2. Extract HI_RMS_CUR from 3-phase currents (Columns 10,11,12 -> index 9,10,11)
            currents = df.iloc[:, 9:12].values
            # Formula: sqrt(mean(sum of squares)) across the cycle
            hi_rms_cur = np.sqrt(np.mean(currents**2))
            
            # 3. Extract HI_RMS_VOL from 3-phase voltages (Columns 14,15,16 -> index 13,14,15)
            voltages = df.iloc[:, 13:16].values
            hi_rms_vol = np.sqrt(np.mean(voltages**2))
            
            trajectory_features.append([hi_max, hi_rms_cur, hi_rms_vol])
            
        if len(trajectory_features) > 0:
            all_trajectories.append(np.array(trajectory_features))
            
    return all_trajectories

# ==========================================
# PHASE 2: DATA STRUCTURING (ALGORITHM 1)
# ==========================================
def create_dataset_multi(trajectories, look_back=40):
    """
    Splits trajectories into sliding windows.
    Returns:
      X_max: for the LSTM (only HI_MAX)
      Y_max: target for the LSTM
      X_all: for the Direct Predictor (all 3 features)
      Y_rul: target RUL for the Direct Predictor
    """
    X_max, Y_max, X_all, Y_rul = [], [], [], []
    
    for traj in trajectories:
        # traj shape: [cycles, 3] where col 0 is HI_MAX
        for i in range(len(traj) - look_back):
            # LSTM needs only HI_MAX (feature index 0)
            X_max.append(traj[i : i + look_back, 0])
            Y_max.append(traj[i + look_back, 0])
            
            # Direct Predictor needs ALL features in the window
            X_all.append(traj[i : i + look_back, :])
            # True RUL is the remaining cycles after the window
            Y_rul.append(len(traj) - (i + look_back))
            
    return np.array(X_max), np.array(Y_max), np.array(X_all), np.array(Y_rul)

# ==========================================
# PHASE 3: HYBRID MODEL CONSTRUCTION
# ==========================================
# 3A. Build Recursive LSTM
def build_recursive_lstm(input_shape):
    """LSTM architecture as defined in the 2023 paper."""
    model = Sequential([
        LSTM(250, return_sequences=True, input_shape=input_shape),
        LSTM(250, return_sequences=False),
        Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='mse')
    return model

# 3B. Extract Uncertain RULs (Algorithm 2)
def get_uncertain_ruls(current_sequence, model, thresholds, look_back):
    """
    Forecasts the sequence recursively until it drops below the failure threshold.
    """
    predicted_path = []
    curr_seq = current_sequence.reshape(1, look_back, 1)
    max_horizon = 5000 
    
    for _ in range(max_horizon):
        next_val = model.predict(curr_seq, verbose=0)[0][0]
        predicted_path.append(next_val)
        
        if next_val < np.min(thresholds):
            break
            
        # Shift sequence left and append new prediction
        new_seq = np.append(curr_seq[0][1:], [[next_val]], axis=0)
        curr_seq = new_seq.reshape(1, look_back, 1)
        
    predicted_path = np.array(predicted_path)
    
    # Find crossing points for the set of thresholds
    uncertain_ruls = []
    for thresh in thresholds:
        crossings = np.where(predicted_path < thresh)[0]
        if len(crossings) > 0:
            uncertain_ruls.append(crossings[0])
        else:
            uncertain_ruls.append(len(predicted_path)) 
            
    return np.mean(uncertain_ruls)

# 3C. Build Direct Predictor (Dense / NARX Equivalent)
def build_direct_predictor(input_dim):
    """
    Input dimension will now be larger: 
    (K cycles * 3 features) + 1 estimated RUL feature
    """
    model = Sequential([
        Dense(100, activation='relu', input_dim=input_dim),
        Dense(100, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

# ==========================================
# PHASE 4: CHALLENGE METRICS & SCORING
# ==========================================
def calculate_challenge_metrics_batch(y_true_list, y_pred_list, t_EoL_list, alpha=0.10, beta=1.0):
    """
    Calculates the PHME 2026 Challenge metrics across a batch of independent trajectories[cite: 1322].
    """
    all_scores, all_acc, all_rmse, all_prec, all_ph_norm = [], [], [], [], []
    
    def normalize_metric(m, a=4443.76, b=1.53, c=4443.76, d=0):
        return a / (m**b + c) + d

    for y_true, y_pred, t_EoL in zip(y_true_list, y_pred_list, t_EoL_list):
        T = len(y_true)
        errors = y_pred - y_true
        
        # 1. Accuracy (Acc)
        acc_sum = np.sum(np.exp(-np.abs(errors) / np.maximum(y_true, 1e-5))) 
        Acc = acc_sum / T
        
        # 2. RMSE & 3. Precision (Pre)
        rmse = np.sqrt(np.mean(errors**2))
        mean_error = np.mean(errors)
        precision = np.sqrt(np.sum((errors - mean_error)**2) / T)
        
        # 4. Prognostic Horizon (PH)
        lower_bound = y_true - (alpha * t_EoL)
        upper_bound = y_true + (alpha * t_EoL)
        in_bounds = (y_pred >= lower_bound) & (y_pred <= upper_bound)
        
        t_alpha = t_EoL 
        for t in range(T):
            if np.all(in_bounds[t:]): 
                t_alpha = t
                break
                
        PH = t_EoL - t_alpha
        PH_norm = PH / t_EoL
        
        # 5. Normalization & Scoring
        RMSE_norm = normalize_metric(rmse)
        Prec_norm = normalize_metric(precision)
        score = (Acc + Prec_norm + RMSE_norm + (beta * PH_norm)) / (3 + beta)
        
        all_scores.append(score)
        all_acc.append(Acc)
        all_rmse.append(rmse)
        all_prec.append(precision)
        all_ph_norm.append(PH_norm)
        
    return {
        "Global_Score": np.mean(all_scores), 
        "Avg_Accuracy": np.mean(all_acc), 
        "Avg_RMSE": np.mean(all_rmse), 
        "Avg_Precision": np.mean(all_prec), 
        "Avg_PH_norm": np.mean(all_ph_norm)
    }

# ==========================================
# EXECUTION WORKFLOW
# ==========================================
if __name__ == "__main__":
    # 1. Load Data
    train_trajectories = load_and_extract_his("../Train/")
    
    K = 40 # Look-back window
    
    # 2. Extract specific arrays for the two models
    X_max, y_max, X_all, y_rul = create_dataset_multi(train_trajectories, K)
    
    # Reshape X_max for LSTM [samples, time steps, features]
    X_max = X_max.reshape((X_max.shape[0], X_max.shape[1], 1))
    
    # 3. Train the Recursive Predictor (LSTM) on HI_MAX only
    recursive_model = build_recursive_lstm((K, 1))
    print("Training Recursive LSTM...")
    recursive_model.fit(X_max, y_max, epochs=2, batch_size=64, verbose=1)
    
    # 4. Prepare Fused Data for the Direct Predictor
    # Determine the range of EoL thresholds
    final_positions = [traj[-1, 0] for traj in train_trajectories] # Col 0 is HI_MAX
    threshold_range = np.arange(np.min(final_positions), 25, 1)
    
    X_direct_fused = []
    y_direct = []
    
    print("Extracting Uncertain RULs and fusing features...")
    # Loop over trajectories with a step to save processing time
    for traj in train_trajectories:
        for i in range(0, len(traj) - K, 100): 
            # The HI_MAX sequence for the LSTM to forecast
            current_max_seq = traj[i : i+K, 0]
            
            # The full feature window (K x 3) flattened for the Direct Predictor
            full_window_flattened = traj[i : i+K, :].flatten()
            
            # True remaining cycles
            true_rul = len(traj) - (i + K)
            
            # Predict uncertain RUL using the trained LSTM
            estimated_rul = get_uncertain_ruls(current_max_seq, recursive_model, threshold_range, K)
            
            # FUSION: [Flattened HI_MAX, HI_RMS_CUR, HI_RMS_VOL] + [Estimated RUL]
            fused_features = np.append(full_window_flattened, estimated_rul)
            X_direct_fused.append(fused_features)
            y_direct.append(true_rul)

    X_direct_fused = np.array(X_direct_fused)
    y_direct = np.array(y_direct)
    
    # 5. Train Direct Predictor (NARX equivalent)
    # input_dim = (40 cycles * 3 features) + 1 RUL feature = 121
    input_dimension = (K * 3) + 1 
    direct_model = build_direct_predictor(input_dim=input_dimension)
    print("Training Fused Direct Predictor...")
    direct_model.fit(X_direct_fused, y_direct, epochs=10, verbose=1)
    
    # 5. Evaluate on Test Data
    print("\nLoading Test Data...")
    test_trajectories = load_and_extract_his("./dataset/Testing/")
    
    test_y_true_list = []
    test_y_pred_list = []
    test_t_EoL_list = []
    
    print("Running inference on test trajectories...")
    for traj in test_trajectories:
        t_EoL_actual = len(traj)
        X_test_fused_traj, y_true_traj = [], []
        
        # Step by 1 for maximum resolution on test predictions
        for i in range(0, t_EoL_actual - K, 100): # Increased step for mock execution speed
            current_max_seq = traj[i : i+K, 0]
            full_window_flattened = traj[i : i+K, :].flatten()
            true_rul = t_EoL_actual - (i + K)
            
            estimated_rul = get_uncertain_ruls(current_max_seq, recursive_model, threshold_range, K)
            fused_features = np.append(full_window_flattened, estimated_rul)
            
            X_test_fused_traj.append(fused_features)
            y_true_traj.append(true_rul)
            
        if len(X_test_fused_traj) > 0:
            X_test_fused_traj = np.array(X_test_fused_traj)
            y_pred_traj = direct_model.predict(X_test_fused_traj, verbose=0).flatten()
            
            test_y_true_list.append(np.array(y_true_traj))
            test_y_pred_list.append(y_pred_traj)
            test_t_EoL_list.append(t_EoL_actual)

    # 6. Calculate Global Score using the Batch Function
    print("\nCalculating Challenge Metrics...")
    metrics = calculate_challenge_metrics_batch(
        y_true_list=test_y_true_list,
        y_pred_list=test_y_pred_list,
        t_EoL_list=test_t_EoL_list
    )

    print("\n==========================================")
    print(f"Final Challenge Score: {metrics['Global_Score']:.4f}")
    print(f"Avg Accuracy:          {metrics['Avg_Accuracy']:.4f}")
    print(f"Avg RMSE:              {metrics['Avg_RMSE']:.4f}")
    print(f"Avg Precision:         {metrics['Avg_Precision']:.4f}")
    print(f"Avg Prognostic Horizon:{metrics['Avg_PH_norm']:.4f}")
    print("==========================================")